In [1]:
%load_ext autoreload
%autoreload 2

import tempfile
import os
import csv

import tensorflow as tf
import numpy as np
import tflite as tfl

import tensorflow.keras.utils
from tensorflow import keras
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import itertools
import soundfile as sf
import librosa
import math
import time
from prune import prune
import tensorflow_model_optimization as tfmot

%load_ext tensorboard
print("done importing")

2023-05-10 13:03:47.160051: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


done importing


In [28]:
# load the model first
# model = hub.load('https://tfhub.dev/google/humpback_whale/1')
interpreter=tf.lite.Interpreter(model_path='./model/quantized_model/quantized_model1.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

i=0
whales_identified=0
whale_samples=0
other_samples_identified=0
other_samples=0
directory_mn='./clips/mn/'
directory_not_mn='./clips/not_mn/'

for file in os.listdir(directory_mn):
    if i==0:
        break
    filename = os.fsdecode(file)
    # if this isn't a .wav file, skip
    if filename[-4:] != '.wav':
        continue
    # try opening the file
    waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(directory_mn+filename))

    waveform = tf.slice(tf.squeeze(tf.expand_dims(waveform, 0),[2]),[1,0],[1,15600])  # makes a batch of size 1

    # Create input tensor out of raw features
    interpreter.set_tensor(input_details[0]['index'], waveform)

    # Run inference
    interpreter.invoke()

    # output_details[0]['index'] = the index which provides the input
    output = interpreter.get_tensor(output_details[0]['index'])

    # If the output type is int8 (quantized model), rescale data
    output_type = output_details[0]['dtype']
    if output_type == np.int8:
        output_scale, output_zero_point = output_details[0]['quantization']
        print("Raw output scores:", output)
        print("Output scale:", output_scale)
        print("Output zero point:", output_zero_point)
        print()
        output = output_scale * (output.astype(np.float32) - output_zero_point)

    # Print the results of inference
    # print("Inference output:", output)
    if output[0][0] >=0.5:
        whales_identified+=1
    whale_samples+=1
    i+=1
labels=[0,0]
for file in os.listdir(directory_not_mn):
    if i == 1000:
        break
    filename = os.fsdecode(file)
    # if this isn't a .wav file, skip
    if filename[-4:] != '.wav':
        continue
        
    # try opening the file
    waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(directory_not_mn+filename))

    waveform = tf.slice(tf.squeeze(tf.expand_dims(waveform, 0),[2]),[0,0],[1,15600])  # makes a batch of size 1

    # Create input tensor out of raw features
    interpreter.set_tensor(input_details[0]['index'], waveform)

    # Run inference
    interpreter.invoke()

    # output_details[0]['index'] = the index which provides the input
    output = interpreter.get_tensor(output_details[0]['index'])

    # If the output type is int8 (quantized model), rescale data
    output_type = output_details[0]['dtype']
    if output_type == np.int8:
        output_scale, output_zero_point = output_details[0]['quantization']
        print("Raw output scores:", output)
        print("Output scale:", output_scale)
        print("Output zero point:", output_zero_point)
        print()
        output = output_scale * (output.astype(np.float32) - output_zero_point)

    # Print the results of inference
    print("Inference output:", output)
    output_probs = tf.math.softmax(output[0])
    pred_label = tf.math.argmax(output_probs)
    labels[pred_label]+=1
    other_samples+=1
    i+=1
print(labels)
print('accuracy: ' + str((whales_identified + other_samples_identified)*1.0 / i))

Inference output: [[0.9588585  0.04114153]]
Inference output: [[0.705622   0.29437795]]
Inference output: [[0.82583016 0.17416981]]
Inference output: [[0.7504094  0.24959058]]
Inference output: [[0.9784589  0.02154106]]
Inference output: [[0.90881956 0.09118046]]
Inference output: [[0.9480487  0.05195133]]
Inference output: [[0.9587386  0.04126145]]
Inference output: [[0.99873704 0.00126295]]
Inference output: [[0.9697845  0.03021552]]
Inference output: [[9.9966431e-01 3.3569723e-04]]
Inference output: [[0.9261284  0.07387164]]
Inference output: [[0.78970975 0.21029031]]
Inference output: [[0.8996983  0.10030168]]
Inference output: [[9.990590e-01 9.409878e-04]]
Inference output: [[0.9770114  0.02298857]]
Inference output: [[9.9959975e-01 4.0027878e-04]]
Inference output: [[0.9950433 0.0049567]]
Inference output: [[0.9689529  0.03104712]]
Inference output: [[0.9090755  0.09092449]]
Inference output: [[0.9626965  0.03730345]]
Inference output: [[9.9981564e-01 1.8432862e-04]]
Inference ou

Inference output: [[0.85077935 0.14922063]]
Inference output: [[0.89283025 0.10716979]]
Inference output: [[0.883092   0.11690808]]
Inference output: [[0.9505581 0.0494419]]
Inference output: [[0.30328077 0.6967192 ]]
Inference output: [[0.9384693  0.06153072]]
Inference output: [[0.99737996 0.00261996]]
Inference output: [[0.9757112  0.02428876]]
Inference output: [[9.9972314e-01 2.7684830e-04]]
Inference output: [[0.99725914 0.00274085]]
Inference output: [[0.9980236 0.0019764]]
Inference output: [[0.92389214 0.07610784]]
Inference output: [[0.9634419  0.03655807]]
Inference output: [[9.9997723e-01 2.2805110e-05]]
Inference output: [[0.95246446 0.04753558]]
Inference output: [[0.98800963 0.01199033]]
Inference output: [[0.90324754 0.09675244]]
Inference output: [[0.8692177  0.13078225]]
Inference output: [[0.9865963  0.01340372]]
Inference output: [[0.937165   0.06283502]]
Inference output: [[0.90521806 0.0947819 ]]
Inference output: [[9.9925846e-01 7.4150023e-04]]
Inference output: 

Inference output: [[0.91491175 0.08508825]]
Inference output: [[0.8570803 0.1429197]]
Inference output: [[0.9541235  0.04587656]]
Inference output: [[9.990086e-01 9.914634e-04]]
Inference output: [[0.99280894 0.00719108]]
Inference output: [[0.91496485 0.08503512]]
Inference output: [[9.99986649e-01 1.34003185e-05]]
Inference output: [[0.92452294 0.07547709]]
Inference output: [[0.9973259  0.00267414]]
Inference output: [[0.93162423 0.0683758 ]]
Inference output: [[0.9951428  0.00485715]]
Inference output: [[9.9985242e-01 1.4760815e-04]]
Inference output: [[0.99878937 0.00121067]]
Inference output: [[0.9909712  0.00902879]]
Inference output: [[0.9975688 0.0024312]]
Inference output: [[0.8610931 0.1389069]]
Inference output: [[0.930463   0.06953695]]
Inference output: [[0.9986286 0.0013714]]
Inference output: [[9.999275e-01 7.251478e-05]]
Inference output: [[0.99040574 0.00959424]]
Inference output: [[0.8928317  0.10716831]]
Inference output: [[9.991027e-01 8.973740e-04]]
Inference outp

Inference output: [[0.9529625  0.04703746]]
Inference output: [[0.9517334  0.04826656]]
Inference output: [[0.9475866  0.05241338]]
Inference output: [[0.94027203 0.05972796]]
Inference output: [[9.9974400e-01 2.5603743e-04]]
Inference output: [[0.97709334 0.02290665]]
Inference output: [[0.94544095 0.05455905]]
Inference output: [[0.99847895 0.001521  ]]
Inference output: [[0.9120125  0.08798746]]
Inference output: [[0.91612667 0.08387335]]
Inference output: [[0.9694857  0.03051432]]
Inference output: [[9.999144e-01 8.561125e-05]]
Inference output: [[0.8323967  0.16760334]]
Inference output: [[0.91531885 0.08468119]]
Inference output: [[0.94497687 0.05502313]]
Inference output: [[0.97187424 0.02812575]]
Inference output: [[0.8834648  0.11653522]]
Inference output: [[9.995042e-01 4.958296e-04]]
Inference output: [[0.9492368  0.05076321]]
Inference output: [[0.8931898  0.10681017]]
Inference output: [[0.93583107 0.06416895]]
Inference output: [[0.85842973 0.14157023]]
Inference output: 

Inference output: [[0.89599556 0.10400447]]
Inference output: [[0.9350207  0.06497931]]
Inference output: [[0.99127126 0.00872879]]
Inference output: [[0.9980008  0.00199917]]
Inference output: [[0.9881009  0.01189908]]
Inference output: [[0.99316746 0.00683257]]
Inference output: [[0.9515106 0.0484894]]
Inference output: [[0.9965581 0.0034419]]
Inference output: [[0.86856765 0.13143232]]
Inference output: [[0.9339297  0.06607027]]
Inference output: [[0.8590773  0.14092273]]
Inference output: [[9.9988246e-01 1.1754581e-04]]
Inference output: [[0.89371324 0.10628678]]
Inference output: [[9.9944395e-01 5.5601407e-04]]
Inference output: [[9.994747e-01 5.253627e-04]]
Inference output: [[0.9057182  0.09428176]]
Inference output: [[0.99828964 0.00171036]]
Inference output: [[9.999732e-01 2.677321e-05]]
Inference output: [[0.9037668  0.09623325]]
Inference output: [[0.9084894  0.09151062]]
Inference output: [[0.9979825  0.00201747]]
Inference output: [[0.9365017  0.06349836]]
Inference output

Inference output: [[0.986105   0.01389503]]
Inference output: [[0.99731594 0.00268405]]
Inference output: [[9.9977225e-01 2.2775475e-04]]
Inference output: [[9.998354e-01 1.645704e-04]]
Inference output: [[0.9749946  0.02500533]]
Inference output: [[0.9045143  0.09548569]]
Inference output: [[9.9958795e-01 4.1206367e-04]]
Inference output: [[0.9982967  0.00170336]]
Inference output: [[0.9608332  0.03916675]]
Inference output: [[9.99884605e-01 1.15378316e-04]]
Inference output: [[0.99781466 0.00218533]]
Inference output: [[9.9974269e-01 2.5726188e-04]]
Inference output: [[0.9893658  0.01063422]]
Inference output: [[0.47933868 0.52066135]]
Inference output: [[0.93865055 0.06134947]]
Inference output: [[0.9964014  0.00359856]]
Inference output: [[9.9995208e-01 4.7940164e-05]]
Inference output: [[0.90404826 0.09595172]]
Inference output: [[0.74721926 0.25278077]]
Inference output: [[0.9923758  0.00762423]]
Inference output: [[0.9858824  0.01411758]]
Inference output: [[0.87031424 0.1296857

In [24]:
print('num whale samples seen: ' + str(whale_samples))
print('num other samples seen: ' + str(other_samples))

num whale samples seen: 0
num other samples seen: 0


In [19]:
i=0
whales_identified=0
whale_samples=0
other_samples_identified=0
other_samples=0
directory_mn='./clips/mn/'
directory_not_mn='./clips/not_mn/'

start_time=time.time()
for file in os.listdir(directory_mn):
    filename = os.fsdecode(file)
    # if this isn't a .wav file, skip
    if filename[-4:] != '.wav':
        continue
    # try opening the file
    try:
        waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(directory_mn+filename))
    except:
        print("error encountered while processing "+filename+", continuing with inference")
        continue
    waveform = tf.expand_dims(waveform,0)  # makes a batch of size 1
    try:
        score = m.predict(waveform)
    except:
        print('could not run inference on ' + filename)
        continue
    if len(score[0]) == 0:
        continue
    score = score[0][0]
    if filename[0:2]=='Mn':
        whale_samples+=1
        if score >= 0.5:
            whales_identified+=1
    else:
        other_samples+=1
        if score < 0.5:
            other_samples_identified+=1
    i+=1
    
for file in os.listdir(directory_not_mn):
    filename = os.fsdecode(file)
    # if this isn't a .wav file, skip
    if filename[-4:] != '.wav':
        continue
        
    # try opening the file
    try:
        waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(directory_not_mn+filename))
    except:
        print("error encountered while processing "+filename+", continuing with inference")
        continue
    waveform = tf.expand_dims(waveform,0)
    try:
        score = m.predict(waveform)
    except:
        print('could not run inference on ' + filename)
        continue
    if len(score[0]) == 0:
        continue
    score = score[0][0]
    if filename[0:2]=='Mn':
        whale_samples+=1
        if score >= 0.5:
            whales_identified+=1
    else:
        other_samples+=1
        if score < 0.5:
            other_samples_identified+=1
    i+=1
        

end_time = time.time()
print('done processing data, elapsed time: '+str(end_time-start_time))

1/1 [==============================] - 0s 86ms/step
could not run inference on Mn-78670550.0-Hawaii_K_14_121216_190000.df20.x.wav
1/1 [==============================] - 0s 60ms/step


2023-05-07 17:15:31.946070: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices = 0 is not in [0, 0)
	 [[{{node GatherV2}}]]


1/1 [==============================] - 0s 71ms/step
could not run inference on Mn-78479588.0-Hawaii_K_14_121216_190000.df20.x.wav
1/1 [==============================] - 0s 77ms/step


2023-05-07 17:15:42.716728: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices = 0 is not in [0, 0)
	 [[{{node GatherV2}}]]


1/1 [==============================] - 0s 75ms/step
could not run inference on Mn-80131491.0-Hawaii_K_14_121216_190000.df20.x.wav
1/1 [==============================] - 0s 73ms/step


...

1/1 [==============================] - 0s 65ms/step
could not run inference on Other-120165645.0-Hawaii_K_20_320_150706_103140_df32.x.wav
1/1 [==============================] - 0s 64ms/step


2023-05-07 17:20:57.250531: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices = 0 is not in [0, 0)
	 [[{{node GatherV2}}]]


1/1 [==============================] - 0s 75ms/step
could not run inference on Other-118523609.0-Hawaii_K_20_320_150801_174730_df32.x.wav
1/1 [==============================] - 0s 65ms/step


2023-05-07 17:20:57.607332: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices = 0 is not in [0, 0)
	 [[{{node GatherV2}}]]


1/1 [==============================] - 0s 70ms/step
done processing data, elapsed time: 365.90700817108154


In [29]:
print('model accuracy: ' + str((whales_identified+other_samples_identified)*1.0/i))
print('proportion of whale samples to total samples: '+str((whale_samples*1.0)/i))
print('total samples: '+ str(i))

model accuracy: 0.46102598267821454
proportion of whale samples to total samples: 0.9733510992671552
total samples: 3002


In [43]:
'''directory='./clips/'
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    # if this isn't a .wav file, skip
    if filename[-4:] != '.wav':
        continue
    if filename[0:2]=='Mn':
        os.system('mv ' + directory + filename + ' ./clips/mn')
    else:
        os.system('mv ' + directory + filename + ' ./clips/not_mn')
'''

train_ds, val_ds = tensorflow.keras.utils.audio_dataset_from_directory(
    directory='./clips',
    batch_size=64,
    validation_split=0.2,
    seed=0,
    output_sequence_length=39124,
    subset='both')

label_names = np.array(train_ds.class_names)
print()
print("label names:", label_names)

Found 35724 files belonging to 2 classes.
Using 28580 files for training.
Using 7144 files for validation.

label names: ['mn' 'not_mn']


In [45]:
def apply_pruning_to_dense(layer):
    if isinstance(layer, tf.keras.layers.Dense):
        return tfmot.sparsity.keras.prune_low_magnitude(layer)
    return layer

m_for_pruning = tf.keras.models.clone_model(
    m,
    clone_function=apply_pruning_to_dense
)

def get_prunable_weights(self):
    return self.get_weights()

# hub.keras_layer.KerasLayer.get_prunable_weights = get_prunable_weights
# print(m_for_pruning.layers[0])
# for layer in m_for_pruning.layers:
    # layer.get_prunable_weights = get_prunable_weights.__get__(layer)

pruned_m=prune(m_for_pruning, train_ds, val_ds)

m.summary()
pruned_m.summary()

2023-05-10 03:23:31.219511: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,?,?]
	 [[{{node Placeholder}}]]
2023-05-10 03:23:31.660306: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [28580]
	 [[{{node Placeholder/_4}}]]
2023-05-10 03:23:31.660941: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [28580]
	 [[

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1)                 23538753  
                                                                 
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 23,538,755
Trainable params: 23,477,443
Non-trainable params: 61,312
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1)                 23538753  
                                                                 
 prune_low_magnitude_dense (  (None, 1)                5         
 PruneLowMagnitude)                                              
                   

In [46]:
m_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1)                 23538753  
                                                                 
 prune_low_magnitude_dense (  (None, 1)                5         
 PruneLowMagnitude)                                              
                                                                 
Total params: 23,538,758
Trainable params: 23,477,443
Non-trainable params: 61,315
_________________________________________________________________


In [47]:
result=m_for_pruning.evaluate(val_ds)

112/112 [==============================] - 185s 2s/step - loss: 5105954.5000 - accuracy: 0.8291


In [48]:
result2=pruned_m.evaluate(val_ds)

112/112 [==============================] - 197s 2s/step - loss: 5105954.5000 - accuracy: 0.8291


In [9]:
help(tf.lite.Interpreter)

Help on class Interpreter in module tensorflow.lite.python.interpreter:

class Interpreter(builtins.object)
 |  Interpreter(model_path=None, model_content=None, experimental_delegates=None, num_threads=None, experimental_op_resolver_type=<OpResolverType.AUTO: 0>, experimental_preserve_all_tensors=False)
 |  
 |  Interpreter interface for running TensorFlow Lite models.
 |  
 |  Models obtained from `TfLiteConverter` can be run in Python with
 |  `Interpreter`.
 |  
 |  As an example, lets generate a simple Keras model and convert it to TFLite
 |  (`TfLiteConverter` also supports other input formats with `from_saved_model`
 |  and `from_concrete_function`)
 |  
 |  >>> x = np.array([[1.], [2.]])
 |  >>> y = np.array([[2.], [4.]])
 |  >>> model = tf.keras.models.Sequential([
 |  ...           tf.keras.layers.Dropout(0.2),
 |  ...           tf.keras.layers.Dense(units=1, input_shape=[1])
 |  ...         ])
 |  >>> model.compile(optimizer='sgd', loss='mean_squared_error')
 |  >>> model.fit